# Using the Simulation Archive to restart a simulation
The Simulation Archive (SA) is a binary file that can be used to restart a simulation. This can be useful when running a long simulation. REBOUND can restart simulation *exactly* (bit by bit) when using a SA. There are some restriction to when a SA can be used. Please read the corresponding paper (Rein & Tamayo 2017) for details. 

We first setup a simulation in the normal way. 

In [1]:
import rebound

sim = rebound.Simulation()
sim.integrator = "whfast"
sim.dt = 2.*3.1415/365.*6 # 6 days in units where G=1
sim.add(m=1.)
sim.add(m=1e-3,a=1.)
sim.add(m=5e-3,a=2.25)
sim.move_to_com()

We then initialize the SA and specify the output filename and output cadence. We can choose the output interval to either correspond to constant intervals in walltime (in seconds) or simulation time. Here, we choose walltime. To choose simulation time instead replace the `interval_walltime` argument with `interval`.

In [2]:
sim.initSimulationArchive("simulationarchive.bin", interval_walltime=1.)

Now, we can run the simulation forward in time. 

In [3]:
sim.integrate(2e5)

Depending on how fast your computer is, the above command may take a couple of seconds. Once the simulation is done, we can delete it from memeory and load it back in from the SA. You could do this at a later time. Note that this will even work if the SA file was generated on a different computer with a different operating system and even a different version of REBOUND. See Rein & Tamayo (2017) for a full discussion on how machine independent code.

In [10]:
sim = None
sim = rebound.Simulation.from_archive("simulationarchive.bin")
print("Time after loading simulation %.1f" %sim.t)

Time after loading simulation 110297.3


If we want to integrate the simulation further in time and append snapshots to the same SA, then we need to set the filename (this is fail safe mechanism to avoid accidentally modifying a SA file):

In [11]:
sim.simulationarchive_filename = "simulationarchive.bin"

Now, let's integrate the simulation further in time. 

In [12]:
sim.integrate(sim.t+2e5)

If we repeat the process, one can see that the SA binary file now includes the new snapshots from the restarted simulation.

In [13]:
sim = None
sim = rebound.Simulation.from_archive("simulationarchive.bin")
print("Time after loading simulation %.1f" %sim.t)

Time after loading simulation 221984.4


A few things to note when restarting a simulation from a SA: 
- If you used any additional forces or post-timestep modifications in the original simulation, then those need to be restored after loading a simulation from a SA.
- If you use the symplectic WHFast integrator with the safe mode turned off, then the simulation will be in an unsychronized state after reloading it. If you want to generate an output, then the simulation needs to be synchronized beforehand. See the WHFast tutorial on how to do that.
- For reproducibility, the SimulationArchive does not output snapshots at the *exact* intervals specified, but rather at the timestep in the integration directly following each interval. This means that if you load from a SimulationArchive and want to reproduce the state in a snapshot later on, you have to pass `exact_finish_time=0` in a call to `sim.integrate`.